<a href="https://colab.research.google.com/github/Psousace/Fundos_Imobiliarios/blob/main/Dividendos_Fiis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

class Fundamentus:

    def __init__(self,Ftype:int,Ativo:str):
        """
        :param Ftype: Tipo de Busca (1: informações gerais, 2:Dividendos)
        :param Ativo: Nome do ativo (Colocar tudo em maiusculo, ex: 'PETR4')
        """
        self.Ftype=Ftype
        self.Ativo=Ativo

    def Get_information(self):
        """
            Faz um request no fundamentus e retorna as informações do ativo.

        :return: Retorna um dataset com o tipo de busca (Dividendos ou Informações Gerais)
        """
        F21=0
        "Buscar info gerais"
        if self.Ftype==1:

            try:
                headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36 OPR/82.0.4227.50'}
                source = requests.get(f"https://fundamentus.com.br/detalhes.php?papel={self.Ativo}",headers=headers).content

                soup = BeautifulSoup(source, 'html.parser')
                contents=soup.find_all(attrs={'class':'label'})
                contents2=soup.find_all(attrs={'class':'data'})
                Fund=pd.DataFrame()
                for x in range(len(contents)):
                    label=contents[x].text
                    label=label.replace("?",'')
                    data=contents2[x].text
                    data=data.replace("\n-", "")
                    data = data.replace("\n", "")
                    data_row=pd.DataFrame({"Info":label,"Data":data},index=[0])
                    Fund=Fund.append(data_row)

                Fund=Fund.reset_index(drop=True)
                Fund["Duplicates"]=Fund["Info"].duplicated()
                F=Fund[Fund["Duplicates"]==True]
                F21=Fund
                FEnd=pd.DataFrame()
                for x in F["Info"]:
                    F21 = F21[(F21["Info"] !=x)]
                    F2 = Fund[(Fund["Duplicates"] == False)&(Fund["Info"]==x)]
                    F23 = Fund[(Fund["Duplicates"] == True) & (Fund["Info"] == x)]

                    F2["Info"]=f"Relatorio 12 meses {x}"
                    F23["Info"] = f"Relatorio 3 meses {x}"

                    FEnd=pd.concat([F2,F23,FEnd])

                F21=F21.append(FEnd).reset_index(drop=True)
                F21=F21.drop(columns=["Duplicates"])
                F21=F21[F21["Info"]!=""].reset_index(drop=True)
            except:
                print("Não foi possível buscar dados do Fundamentus!")

        "Buscar dividendos"
        if self.Ftype == 2:
            headers = {
                'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36 OPR/82.0.4227.50'
            }
            source = requests.get(f"https://fundamentus.com.br/proventos.php?papel={self.Ativo}&tipo=2",headers=headers).content
            soup = BeautifulSoup(source, 'html.parser')
            teste = soup.find_all("h1")[0].text
            prove="Go"
            if teste=="Nenhum provento encontrado ":
                prove="no"

                source = requests.get(f"https://fundamentus.com.br/fii_proventos.php?papel={self.Ativo}&tipo=2",headers=headers).content
                soup = BeautifulSoup(source, 'html.parser')
                teste = soup.find_all("h1")[0].text
                if teste == "Nenhum provento encontrado ":
                    prove = "No"
                else:
                    prove="Go"
                    if prove == "Go":
                        contents = soup.find_all("td")
                        # contents2 = soup.find_all(attrs={'class': 'data'})
                        Fund = pd.DataFrame()
                        g = 0
                        Divi = pd.DataFrame()

                        for x in contents:

                            g = g + 1
                            if g == 1:
                                data = x.text
                            if g == 2:
                                tipo = x.text
                            if g == 3:
                                data2 = x.text
                            if g == 4:
                                valor = x.text
                                g = 0
                                new_row = pd.DataFrame({"Data": data, "Valor pago": valor, "Tipo de dividendos": tipo,"Data de pagamento": data2}, index=[0])
                                Divi = Divi.append(new_row)
                                F21=Divi
            else:
                if prove=="Go":
                    contents = soup.find_all("td")

                    #contents2 = soup.find_all(attrs={'class': 'data'})
                    Fund = pd.DataFrame()
                    g=0
                    Divi=pd.DataFrame()
                    for x in contents:

                        g=g+1
                        if g==1:
                            data=x.text
                        if g==2:
                            valor = x.text
                        if g==3:
                            tipo=x.text
                        if g==4:
                            data2 = x.text
                        if g==5:
                            qntd = x.text
                            g=0
                            new_row=pd.DataFrame({"Data":data,"Valor pago":valor,"Tipo de dividendos":tipo,"Data de pagamento":data2,"Qauntidade de Ações":qntd},index=[0])
                            Divi=Divi.append(new_row)
                            F21 = Divi

        return F21

    def save(self,data):
        """
            Salva o Dataset.

        :param data: Dataset para salvar
        """
        self.data=data
        if self.Ftype==1:
            data.to_csv(f"Relatório de Bignumbers {self.Ativo}.csv", sep=";", index=False, encoding='latin-1', mode="w",decimal=',')
        if self.Ftype==2:
            data.to_csv(f"Relatório de Dividendos {self.Ativo}.csv", sep=";", index=False, encoding='latin-1', mode="w",decimal=',')



In [4]:
D=Fundamentus(2,"HGLG11")
D1=Fundamentus(1,"HGLG11")
Div=D.Get_information()
Inf=D1.Get_information()
D.save(Div)
D1.save(Inf)

In [ ]:
display(Div)

,Data,Valor pago,Tipo de dividendos,Data de pagamento
0,31/08/2023,"0,78",Rendimento,15/09/2023
0,31/07/2023,"0,78",Rendimento,14/08/2023
0,30/06/2023,"0,79",Rendimento,14/07/2023
0,31/05/2023,"0,75",Rendimento,15/06/2023
0,28/04/2023,"0,74",Rendimento,15/05/2023
...,...,...,...,...
0,31/10/2018,"0,60",Rendimento,16/11/2018
0,28/09/2018,"0,70",Rendimento,15/10/2018
0,31/08/2018,"0,50",Rendimento,17/09/2018
0,31/07/2018,"0,13",Rendimento,14/08/2018
